In [1]:
import pandas as pd
import numpy as np
import csv
from pathlib import Path 
import re
import string

In [2]:
df= pd.read_csv(r'/Users/lolitalolita/Downloads/recipes.csv')

In [3]:
# Get healthy and vegan recipes 

df=df[df['RecipeCategory'].isin(['Healthy','Vegan'])]


In [4]:
# Get calories and content per one serving
df_per_serving=df[['Calories','CholesterolContent','CarbohydrateContent','SugarContent','ProteinContent']].div(df.RecipeServings, axis=0) 

In [5]:
df['RecipeCategory'].unique()

array(['Vegan', 'Healthy'], dtype=object)

In [6]:
df

,RecipeId,Name,RecipeCategory,Calories,CholesterolContent,CarbohydrateContent,SugarContent,ProteinContent,RecipeServings,HighScore
195,236464,Sweet and Sour Cucumber Salad,Vegan,52.1,0.0,12.5,6.2,2.1,2,1
496,512214,Gluten Free Vegan Zucchini Eggplant Lasagna,Vegan,223.1,2.2,29.6,15.8,12.1,8,1
796,455919,Ginger Lime Fish Parcels,Healthy,114.4,61.6,6.5,4.7,17.3,4,0
847,94335,Salmon With Chili-honey Glaze,Healthy,232.9,78.8,15.2,13.8,30.5,4,1
1008,201755,Tibetan Style Seitan Burritos,Vegan,195.5,0.0,28.4,6.3,4.2,6,0
...,...,...,...,...,...,...,...,...,...,...
41689,450356,Marie's Tilapia Marinara,Healthy,745.6,178.2,89.0,6.7,51.7,2,1
42779,472204,America's Test Kitchen Thin-Crust Pizza,Healthy,184.5,12.8,25.6,2.0,7.4,16,1
42847,503736,3 Ingredient Individual Quinoa Pizza Crust (No...,Healthy,297.2,0.0,48.9,0.1,10.0,1,1
42927,127988,Ww 4 Points - Cinnamon Pepper Shrimp,Healthy,197.0,172.8,9.8,3.8,24.7,2,0


In [7]:
# Import PuLP modeler functions
from pulp import *

In [8]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("The Food Emissions Problem", LpMinimize)

/Users/lolitalolita/opt/anaconda3/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [9]:
Name_dict = list(df['Name'])

Calories_dict = dict(zip(Name_dict, df_per_serving['Calories']))
CholesterolContent_dict = dict(zip(Name_dict , df_per_serving['CholesterolContent']))
CarbohydrateContent_dict = dict(zip(Name_dict , df_per_serving['CarbohydrateContent']))
SugarContent_dict = dict(zip(Name_dict, df_per_serving['SugarContent']))
ProteinContent_dict = dict(zip(Name_dict, df_per_serving['ProteinContent']))

In [10]:
# A dictionary called 'ingredient_vars' is created to contain the referenced Variables
recipe_name_vars = LpVariable.dicts("R", Name_dict, 0)

In [11]:
# The objective function is added to 'prob' first
prob += (
    lpSum([SugarContent_dict[i] * recipe_name_vars[i] for i in Name_dict]),
    "Total sugar content per dish",)

In [12]:
# The constraints are added to 'prob'
#prob += lpSum([recipe_name_vars[i] for i in Name_dict]) == 100, "PercentagesSum"
prob += lpSum([recipe_name_vars[i] for i in Name_dict]) 

prob += (
    lpSum([Calories_dict[i] * recipe_name_vars[i] for i in Name_dict]) <= 2000.0,
    "Calories_dict_Requirement_max",
)
prob += (
    lpSum([Calories_dict[i] * recipe_name_vars[i] for i in Name_dict]) >= 1500.0,
    "Calories_dict_Requirement_min",
)


prob += (
    lpSum([CholesterolContent_dict[i] * recipe_name_vars[i] for i in Name_dict]) <= 13.0,
    "CholesterolContent_Requirement_max",
)
prob += (
    lpSum([CholesterolContent_dict[i] * recipe_name_vars[i] for i in Name_dict]) >= 5.0,
    "CholesterolContent_Requirement_min",
)

prob += (
    lpSum([CarbohydrateContent_dict[i] * recipe_name_vars[i] for i in Name_dict]) <= 150.0,
    "CarbohydrateContent_Requirement_max",
)

prob += (
    lpSum([CarbohydrateContent_dict[i] * recipe_name_vars[i] for i in Name_dict]) >= 120.0,
    "CarbohydrateContent_Requirement_min",
)

prob += (
    lpSum([ProteinContent_dict[i] * recipe_name_vars[i] for i in Name_dict]) <= 112.0,
    "ProteinContent_Requirement_max",
)
prob += (
    lpSum([ProteinContent_dict[i] * recipe_name_vars[i] for i in Name_dict]) >= 70.0,
    "ProteinContent_Requirement_min",
)

/Users/lolitalolita/opt/anaconda3/lib/python3.9/site-packages/pulp/pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [13]:
# The problem data is written to an .lp file
prob.writeLP("RecipeModel.lp")

[R_3_2_1_Shrimp,
 R_3_Ingredient_Individual_Quinoa_Pizza_Crust_(No_Soaking!),
 R_Air_Fryer_Crispy_Panko_Shrimp,
 R_America's_Test_Kitchen_Thin_Crust_Pizza,
 R_Apple_Butter_Pie,
 R_Asian_Sesame_Cod,
 R_Azerbaijani_Pomegranate_and_Spinach_Soup_(Vegan),
 R_BBQ_Pork_Fried_Rice,
 R_Baked_Orange_Salmon,
 R_Baked_Pasta_With_Sausage,_Tomatoes,_and_Cheese,
 R_Baked_Pesto_Pasta_With_Tomato_&_Cheese,
 R_Baked_Salmon_Fish_Sticks,
 R_Barbecued_Salmon__and__Easy_Marinade,
 R_Bean,_Tomato,_&amp;_Butternut_Squash_Soup,
 R_Beans_Bourguignon,
 R_Beer_Batter,
 R_Best_Ever_Spicy_Boiled_Shrimp,
 R_Butternut_Ginger_Soup,
 R_Caramel_Sweetened_Condensed_Milk,
 R_Caraway_Noodles,
 R_Carne_Apache,
 R_Carrot_Bisque,
 R_Carrot_and_Cashew_Nut_Roast,
 R_Chicken,_Potato_and_Vegetable_Bake,
 R_Chicken,_Shrimp,_and_Ham_Jambalaya,
 R_Chicken__Sweet_and_Spunky,
 R_Chicken_and_Rice_Casserole,
 R_Chinese_Buffet_Black_Pepper_Chicken,
 R_Cream_of_Mushroom_Soup_(Raw_Food),
 R_Creamy_Carrot_Rice,
 R_Creamy_Garlic_Shrimp,
 R_C

In [14]:
# The problem is solved using PuLP's choice of Solver
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/lolitalolita/opt/anaconda3/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/j9/7y5q0v6x4p3f2fdr5f5ffl0m0000gn/T/b7374b53795e496da09e88ff72976275-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/j9/7y5q0v6x4p3f2fdr5f5ffl0m0000gn/T/b7374b53795e496da09e88ff72976275-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 1493 RHS
At line 1502 BOUNDS
At line 1503 ENDATA
Problem MODEL has 8 rows, 175 columns and 1304 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 8 (0) rows, 174 (-1) columns and 1304 (0) elements
Perturbing problem by 0.001% of 708449.72 - largest nonzero change 0.00070767963 ( 0.0056623223%) - largest zero change 0
0  Obj 0 Primal inf 0.7318967 (4)
7  Obj 3.97507
Optimal - objective value 3.9750448
After Postsolve, objective 

1

In [15]:
# The status of thesolution is printed to the screen
print("Status:", LpStatus[prob.status])

Status: Optimal


In [16]:
# Each of the variables is printed with it's resolved optimum value
    
for v in prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

R_Baked_Orange_Salmon = 0.33591731
R_Cream_of_Mushroom_Soup_(Raw_Food) = 1.9541966
R_Flax_Quinoa_Crackers_(Gluten_Free) = 1.6849309


In [17]:
# The optimised objective function value is printed to the screen
print("Minimal sugar intake = ", value(prob.objective))

Minimal sugar intake =  3.97504481
